In [129]:
from scipy.stats import skew
from IPython.display import display
import matplotlib.pyplot as plt
import sys
sys.path.append('/Users/yanhuan/anaconda/pkgs/seaborn-0.7.1-py27_0/lib/python2.7/site-packages')
import seaborn as sns
import numpy as np
import pandas as pd


SP500=pd.read_csv('SP500_new.csv')
Nasdaq=pd.read_csv('Nasdaq_new.csv')
DJI=pd.read_csv('DJI_new.csv')
DAX=pd.read_csv('DAX_new.csv')
Paris=pd.read_csv('Paris_new.csv')
Tokyo=pd.read_csv('Tokyo_new.csv')
HongKong=pd.read_csv('HongKong_new.csv')
Aus=pd.read_csv('Aus_new.csv')

Data Preprocess

In [130]:
Target=SP500['Adj Close'].pct_change(1).shift(-1)

In [131]:
datasets=[SP500,Nasdaq,DJI,DAX,Paris,Tokyo,HongKong,Aus]
namesets=['SP500','Nasdaq','DJI','DAX','Paris','Tokyo','HongKong','Aus']

In [132]:
def addFeatures(datasets,DaysRet,DaysRetMovAvg,DaysLags):
    """
    compute dataset's DaysReturns,DaysReturnMovAvg and DaysLags
    DaysReturnMovAvg must >=2
    """
    Max=max(DaysRet,DaysRetMovAvg,DaysLags+1)
    for i in range(len(datasets)):
        dataset=datasets[i]
        nameset=namesets[i]
        for i in range(1,DaysRet+1):
            dataset[nameset+'_'+str(i)+'_'+'DaysRet'] = dataset['Adj Close'].pct_change(i)
        for i in range(2,DaysRetMovAvg+1):
            dataset[nameset+'_'+str(i)+'_'+'DaysRetMovAvg'] = pd.rolling_mean(dataset[nameset+'_'+str(1)+'_'+'DaysRet'], i)
        for i in range(1,DaysLags+1):
            dataset[nameset+'_'+str(i)+'_'+'DaysLags'] = dataset[nameset+'_'+str(1)+'_'+'DaysRet'].shift(i)
        dataset=dataset[Max:]
    return Max

In [134]:
def mergedataset(datasets):
    useless=['Date','Open','High','Low','Close','Adj Close','Volume']
    for i in range(len(datasets)):
        datasets[i]=datasets[i].drop(useless, axis = 1)
    new_dataset = pd.concat(datasets,axis=1)
    return new_dataset

In [3]:
def featureselection(datasets,DaysRet_range,DaysRetMovAvg_range,DaysLags_range):
    """
    this function can be used to tune the parameters of DaysReturns,DaysReturnMovAvg and DaysLags.
    """
    total={}
    for i in DaysRet_range:
        for j in DaysRetMovAvg_range:
            for t in DaysLags_range:
                Max=addFeatures(datasets,i,j,t)
                features=mergedataset(datasets)
                
                features=features[Max:-1]
                Target=Target[Max:-1]
                #transfer Target data to binary
                Target[Target > 0] = 1
                Target[Target <= 0] = 0
                #Shuffle and Split Data
                X_train=features[:-500]
                X_test=features[-500:]
                y_train=Target[:-500]
                y_test=Target[-500:]
                
                model=RandomForestClassifier(n_estimators=120)
                model.fit(X_train, y_train)
                
                y_predict=model.predict(X_test)
                
                total[str(accuracy_score(y_test,y_predict))]=[i,j,t]
    compare=[]
    for i in total.keys():
        compare.append(float(i))
    Best_accu=max(compare)
    print('Best accuracy:',Best_accu)
    print(total[str(Best_accu)])